In [ ]:
import sys
import numpy as np

import matplotlib.pyplot as plt

from pymedphys.gamma import GammaInternalFixedOptions, calculate_min_dose_difference, convert_to_percent_pass

In [ ]:
def get_dummy_gamma_set():
    grid_x = np.arange(0, 1, 0.1)
    grid_y = np.arange(0, 1.2, 0.1)
    grid_z = np.arange(0, 1.4, 0.1)
    dimensions = (len(grid_x), len(grid_y), len(grid_z))
    coords = (grid_x, grid_y, grid_z)

    reference = np.zeros(dimensions)
    reference[3:-2:, 4:-2:, 5:-2:] = 1.015

    evaluation = np.zeros(dimensions)
    evaluation[2:-2:, 2:-2:, 2:-2:] = 1

    expected_gamma = np.zeros(dimensions)
    expected_gamma[2:-2:, 2:-2:, 2:-2:] = 0.4
    expected_gamma[3:-3:, 3:-3:, 3:-3:] = 0.7
    expected_gamma[4:-4:, 4:-4:, 4:-4:] = 1
    expected_gamma[3:-2:, 4:-2:, 5:-2:] = 0.5

    return coords, reference, evaluation, expected_gamma


coords, reference, evaluation, _ = get_dummy_gamma_set()

In [ ]:
def calc_gamma_for_case(distance, threshold_ratio):
    options = GammaInternalFixedOptions.from_user_inputs(
        coords, reference,
        coords, evaluation,
        distance * threshold_ratio, distance, lower_percent_dose_cutoff=0, quiet=True
    )

    still_searching_for_gamma = np.ones_like(
        options.flat_dose_reference).astype(bool)
    current_gamma = np.inf * np.ones_like(options.flat_dose_reference)
    distance = 0.0

    while distance <= options.maximum_test_distance:
        to_be_checked = (
            options.reference_points_to_calc & still_searching_for_gamma)

        min_relative_dose_difference = calculate_min_dose_difference(
            options, distance, to_be_checked)

        gamma_at_distance = np.sqrt(
            (min_relative_dose_difference /
             (options.dose_percent_threshold / 100)) ** 2
            + (distance / options.distance_mm_threshold) ** 2)

        current_gamma[to_be_checked] = np.min(
            np.vstack((
                gamma_at_distance, current_gamma[to_be_checked]
            )), axis=0)

        still_searching_for_gamma = (
            current_gamma > distance / options.distance_mm_threshold)

        if options.skip_once_passed:
            still_searching_for_gamma = (
                still_searching_for_gamma & (current_gamma >= 1))

        distance += options.distance_step_size

        if np.sum(to_be_checked) == 0:
            break

    gamma = np.reshape(
        current_gamma, np.shape(reference))
    gamma[np.isinf(gamma)] = np.nan
    
    return gamma

In [ ]:
base_dist = 0.3
threshold_ratios = np.arange(1,7)

gamma_store = {}
for ratio in threshold_ratios:
    gamma = calc_gamma_for_case(base_dist, ratio)
    gamma_store[ratio] = gamma



# convert_to_percent_pass


In [ ]:
# convert_to_percent_pass(gamma_store[5])

In [ ]:
gamma_scaling = np.arange(1, 5, 0.1)

distance_thresholds = []
dose_thresholds = []
pass_rates = []

for scale in gamma_scaling:
    for threshold_ratio, gamma in gamma_store.items():
        pass_rates.append(convert_to_percent_pass(gamma_store[threshold_ratio] / scale))
        distance_thresholds.append(base_dist * scale)
        dose_thresholds.append(distance_thresholds[-1] * threshold_ratio)
        

plt.scatter(distance_thresholds, dose_thresholds, c=pass_rates, vmin=90, vmax=100, cmap='coolwarm_r')
plt.colorbar()
plt.xlim([0.3, 1.5])
plt.ylim([0.2, 3])

plt.xlabel('Distance Threshold')
plt.ylabel('Dose Threshold')



In [ ]:
# pass_rates

In [ ]:
dose_thresholds

In [ ]:
plt.contourf(threshold_ratios, )

In [ ]:
plt.pcolormesh(gamma[5,:,:])
plt.colorbar()